In [13]:
import time
import serial
import sys
import glob
import serial.tools.list_ports

def getOpenPorts():
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')
    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

def parsePortName(portinfo):
    """
    On macOS and Linux, selects only usbserial options and parses the 8 character serial number.
    """
    portlist = []
    for port in portinfo:
        if sys.platform.startswith('win'):
            portlist.append(port[0])
        elif sys.platform.startswith('darwin') or sys.platform.startswith('linux'):
            if 'usbserial' in port[0]:
                namelist = port[0].split('-')
                portlist.append(namelist[-1])
    return portlist

def find_serial_ports():
    ports = getOpenPorts()
    for port in ports:
        if 'usbserial' in port:
            return port
            
class Connection(object):
    def __init__(self, port, baudrate, x = 0, mode = 0, verbose=True): 

        # verbose = True to get print statements
        # mode 0 is infuse, mode 1 is withdraw
        # no idea what x does
        
        self.port = port
        self.baudrate = baudrate
        self.x = x
        self.mode = mode
        self.verbose = verbose
        self.units = None
        self.diameter = None
        self.volume = None
        self.rate = None
        self.delay = None
        # self.runtime = None

    def openConnection(self):
        try:
            self.ser = serial.Serial()
            self.ser.baudrate = self.baudrate
            self.ser.port = self.port
            self.ser.timeout = 0
            self.ser.open()
            if self.ser.isOpen():
                if self.verbose:
                    print(f"Opened port {self.ser.port}")
                    print(self.ser)
                self.getPumpStatus()
                self.ser.flushInput()
                self.ser.flushOutput()
        except Exception as e:
            if self.verbose:
                print('Failed to connect to pump')
                print(e)
            pass

    def closeConnection(self):
        self.ser.close()
        if self.verbose:
            print("Closed connection")

    def sendCommand(self, command):
        try:
            arg = bytes(str(command), 'utf8') + b'\r'
            self.ser.write(arg)
            time.sleep(0.5)
            response = self.getResponse()
            return response
        except TypeError as e:
            if self.verbose:
                print(e)
            self.ser.close()

    def getResponse(self):
        try:
            response_list = []
            while True:
                response = self.ser.readlines()
                for line in response:
                    line = line.strip(b'\n').decode('utf8')
                    line = line.strip('\r')
                    if self.verbose:
                        print(line)
                    response_list.append(line)
                break
            return response_list
        except TypeError as e:
            if self.verbose:
                print(e)
            self.closeConnection()
        except Exception as f:
            if self.verbose:
                print(f)
            self.closeConnection()

    def constructCommand(self, units, mode, diameter, volume, rate, delay):

        # with hexw2 commands, there is no way to specify the time you want to infuse
        # the volume and flow rate should be chosen according to the desired infusion time
        
        string = f"hexw2 {units} {mode} {diameter} {volume} {rate} {delay} start"
        return string

    def infuse(self):

        # start pump infusion
        
        command = self.constructCommand(
            units=self.units, 
            mode=0,  # 0 for infusion
            diameter=self.diameter, 
            volume=self.volume, 
            rate=self.rate, 
            delay=self.delay
        )
        response = self.sendCommand(command)
        return response

    def withdraw(self):

        # start pump withdrawal
        
        command = self.constructCommand(
            units=self.units, 
            mode=1,  # 1 for withdrawal
            diameter=self.diameter, 
            volume=self.volume, 
            rate=self.rate, 
            delay=self.delay
        )
        response = self.sendCommand(command)
        return response
        
    def startPump(self):

        # start pump (defaults to last used mode–infusion or withdrawal. Preferable to use infuse() or withdraw() instead
        
        command = 'start'
        # command = self.addX(command)
        # command = self.addMode(command)
        response = self.sendCommand(command)
        return response
        print(response)

    def stopPump(self):

        # stop pump
        
        command = 'stop'
        command = self.addX(command)
        response = self.sendCommand(command)
        return response

    def pausePump(self):

        # pause pump
        
        command = 'pause'
        command = self.addX(command)
        response = self.sendCommand(command)
        return response

    def restartPump(self):

        # restart pump–useful after pump has been stopped (run must be started over at that point)
        
        command = 'restart'
        response = self.sendCommand(command)
        return response

    def setUnits(self, units):

        # set units for flow rate
        
        units_dict = {'mL/min': '0', 'mL/hr': '1', 'μL/min': '2', 'μL/hr': '3'}
        command = 'set units ' + units_dict[units]
        response = self.sendCommand(command)
        self.units = units_dict[units]
        return response

    def setDiameter(self, diameter):

        # set syringe diameter
        
        command = 'set diameter ' + str(diameter)
        response = self.sendCommand(command)
        self.diameter = diameter
        return response

    def setRate(self, rate):

        # set volumetric flow rate
        
        if isinstance(rate,list):
            # if list of volumes entered, use multi-step command
            command = 'set rate '+','.join([str(x) for x in rate])
        else:
            command = 'set rate ' + str(rate)
        response = self.sendCommand(command)
        self.rate = rate
        return response

    def setVolume(self, volume):

        # set desired infusion volume

        
        if isinstance(volume,list):
            # if list of volumes entered, use multi-step command
            command = 'set volume '+','.join([str(x) for x in volume])
        else:
            command = 'set volume ' + str(volume)
        response = self.sendCommand(command)
        self.volume = volume
        return response

    def setDelay(self, delay):

        # set start time delay (in minutes)
        
        if isinstance(delay,list):
            # if list of volumes entered, use multi-step command
            command = 'set delay '+','.join([str(x) for x in delay])
        else:
            command = 'set delay ' + str(delay)
        response = self.sendCommand(command)
        self.delay = delay
        return response

    def setTime(self, *timer): # Don't really know if we need this funtion

        # sets target time for a pump run

        if timer:
            command = 'set time ' + str(timer)
            response = self.sendCommand(command)
            self.runtime = timer
            return response
        else:
            self.runtime = self.volume/self.rate
        
    def getParameterLimits(self):

        """
        Returns the min/max values that can be set for transfer volume and transfer rate.
        The returned values are dependent on the syringe inner diameter (ID). The transfer
        volume limits are typically greater than the maximum volume of the syringe being
        used
        """
        
        command = 'read limit parameter'
        response = self.sendCommand(command)
        return response

    def getParameters(self):

        """
        Returns the currently set parameters (Basic Mode only).
        Returns: rate units, syringe inner diameter, transfer rate, priming rate, time (in whole
        number minutes), transfer volume, time delay (in whole number minutes)
        """
        
        command = 'view parameter'
        response = self.sendCommand(command)
        return response

    def getDisplacedVolume(self):

        # Returns the transferred volume for the current run/step
        
        command = 'dispensed volume'
        response = self.sendCommand(command)
        return response

    def getElapsedTime(self):

        # Returns the elapsed time (in minutes) for the current run/step.
        
        command = 'elapsed time'
        response = self.sendCommand(command)
        return response

    def getPumpStatus(self):

        """
        Returns the current status of the pump. The returned integer correlates to the status
        of the pump.
        0: Pump stopped
        1: Pump running
        2: Pump paused
        3: Pump delayed
        4: Pump stalled
        """
        
        command = 'pump status'
        response = self.sendCommand(command)
        return response
        
    def addMode(self, command):

        # designed to add the mode to the command. Doesn't work on Fusion 200 I've found
        if self.mode == 0:
            return command
        else:
            return command + ' ' + str(self.mode)

    def addX(self, command):

        # No idea what x is or does
        
        if self.x == 0:
            return command
        else:
            return str(self.x) + ' ' + command

    def flowLoop(self, loops, infuse_time):

        """
        loop through infuse/withdraw cycles for oscillatory flow
        
        :param loop: interger
            number of times you want to cycle between infuse and withdraw
        
        :param time: float
            length of infusion and withdraw time in seconds
    
        """
    
        # want some functionality to check if the time sleep will be longer than the actual infusion time
        
        if None in [self.units, 
                  self.diameter, 
                  self.volume, 
                  self.rate, 
                  self.delay]:
          
            print("Not all pump parameters are set. Please set units, diameter, volume, rate, and delay.")
            return
    
        units_dict = {'mL/min': '0', 'mL/hr': '1', 'μL/min': '2', 'μL/hr': '3'}
        
        if str(self.units) in ('0','2'):
            if infuse_time  > (self.volume/self.rate*60): # need to account for units of time here. infusion_time is in seconds
                print(f"Error: Provided loop time {infuse_time}s is greater than the pump runtime {self.volume/self.rate*60}s.") 
                return
    
        elif str(self.units) in ('1','3'):
            if infuse_time  > (self.volume/self.rate*3600): # need to account for units of time here. infusion_time is in seconds
                print(f"Error: Provided loop time {infuse_time}s is greater than the pump runtime {self.volume/self.rate*3600}s.") 
                return
            
        try:
            for i in range(loops):
                # change conn to self
                conn.infuse()
                time.sleep(infuse_time)
                conn.pausePump()
                time.sleep(0.5)
                conn.withdraw()
                time.sleep(infuse_time)
                conn.pausePump()
                time.sleep(0.5)
    
        except KeyboardInterrupt:
            conn.pausePump()
            print("Process interrupted by user. Pausing...")

In [14]:
# get open port info
portinfo = getOpenPorts()

# alternatively, can find a usb port directly and plug that into the connection
port = find_serial_ports()

# MUST set baudrate in pump "System Settings", and MUST match this rate:
baudrate = 9600

# initiate Connection object with first open port
conn = Connection(port=port , baudrate=baudrate , x=0 , mode=0, verbose=False) #set verbose to true to see infos

# Open Connection to pump
conn.openConnection()

# Setup parameters for basic run
units = 'μL/min'  # OPTIONS: 'mL/min','mL/hr','μL/min','μL/hr'
diameter = 0.73  # in mm
volume = 5  # according to units
rate = 0.2  # units according to units object
runtime = volume / rate  # this is calculated implictly by pump
delay = 0.0  # 0 second delay (0.5 is 30s delay)

# Communicate parameters to pump
conn.setUnits(units)
conn.setDiameter(diameter)
conn.setVolume(volume)
conn.setRate(rate)
conn.setDelay(delay)


['set delay 0.0', 'delay = 0.00', '>']

In [16]:
conn.flowLoop(3,5.0)

Process interrupted by user. Pausing...
